<a href="https://colab.research.google.com/github/GerardoFesta/3DFER_SE4AI/blob/main/3DTransformation/LandmarkVoxelization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import torch
from torch.utils.data import Dataset


Connection to Google Drive, do not execute if you do not wish to connect it.

In [ ]:
import os
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


The code appears to perform voxelization of an image. It returns a cubic matrix with dimensions 24x24x24, called "cubic_matrix." The x, y, and z coordinates are normalized to fit within the range 0-24. The cubic matrix is then further normalized between 0 and 1.


In [ ]:
def voxelization(img):
  cubic_matrix=np.zeros((24,24,24))
  x_array = img[:, 0]
  y_array = img[:, 1]
  z_array = img[:, 2]
  x= np.array(23.99*(x_array-x_array.min())/(x_array.max()-x_array.min()),dtype=np.int)
  y= np.array(23.99*(y_array-y_array.min())/(y_array.max()-y_array.min()),dtype=np.int)
  z= np.array(23.99*(z_array-z_array.min())/(z_array.max()-z_array.min()),dtype=np.int)
  for i in range(0,len(x)):
    cubic_matrix[x[i],y[i],z[i]]+=1
    cubic_matrix[:,:,:]/=cubic_matrix[:,:,:].max() #DATI GIA' NORMALIZZATI TRA ZERO E UNO
  return cubic_matrix


Loading training and test data.

In [ ]:
training_path="drive/MyDrive/training_set.csv"
test_path ="drive/MyDrive/test_set.csv"
train_df=pd.read_csv(training_path)
test_df=pd.read_csv(test_path)





label_dict = {"angry":0, "sad": 1, "neutral": 2, "surprise": 3, "disgust": 4, "fear": 5, "happy": 6}


test_df['landmarks'] = test_df['landmarks'].apply(lambda landmark: eval(landmark))

train_df['landmarks'] = train_df['landmarks'].apply(lambda landmark: eval(landmark))

test_df['label'] = test_df['label'].apply(lambda lab: label_dict[lab])

train_df['label'] = train_df['label'].apply(lambda lab: label_dict[lab])

array_train = train_df['landmarks'].to_numpy()
X_train = np.stack([np.array(lst) for lst in array_train])
y_train = train_df['label'].to_numpy()
array_test = test_df['landmarks'].to_numpy()
X_test = np.stack([np.array(lst) for lst in array_test])
y_test = test_df['label'].to_numpy()


In [ ]:
del train_df
del test_df

Voxelization of training and test data and save the results in npz files

In [ ]:
print(X_train.shape)
X_cubic_data_train= np.zeros(shape=(26705, 24, 24, 24))
i=0
for single_image in X_train:
  cubic_matrix = voxelization(single_image)
  X_cubic_data_train[i] = cubic_matrix
  i+=1

np.savez('train_dataset.npz', data=X_cubic_data_train, labels=y_train)
del X_cubic_data_train

X_cubic_data_test = np.zeros(shape=(6678, 24, 24, 24))

i=0
for single_image in X_test:
  cubic_matrix = voxelization(single_image)
  X_cubic_data_test[i] = cubic_matrix
  i+=1

np.savez('test_dataset.npz', data=X_cubic_data_test, labels=y_test)



(26705, 478, 3)


<ipython-input-2-c83277b54535>:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x= np.array(23.99*(x_array-x_array.min())/(x_array.max()-x_array.min()),dtype=np.int)
<ipython-input-2-c83277b54535>:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additi